# PublicDatasets

## 1. Introduction

### 1.1 Info

**Current Pipeline:**

* First pick targets
* Find appropriate input
* Run Downloader
* Read PDFs
* Query for Relations
* Save Results

#### 1.1.1 Target Venues

* MIDL 2021
  * Proceedings to use: https://proceedings.mlr.press/v143/

In [1]:
venues = ['MIDL 2021',
          'CHIL 2021',
          'ML4H 2020',
          'ML4H 2021'
         ]

In [2]:
special_separator = '___'

#### 1.1.2 Target Datasets

* [Data Science Bowl 2017](https://www.kaggle.com/c/data-science-bowl-2017)
  * See here for literature: https://scholar.google.com/scholar?hl=en&as_sdt=0%2C5&q=%22data+science+bowl%22+%2B+lung&btnG=

### 1.2 Imports

In [3]:
import os,sys
import scrapy
import pdfminer
import pandas as pd

In [4]:
import warnings
warnings.filterwarnings('ignore')

import logging
logging.getLogger("scrapy").setLevel(logging.CRITICAL)
logging.getLogger("pdfminer").setLevel(logging.CRITICAL)

### 1.3 Misc. Variables

In [5]:
pdfs=[]
texts=[]

#dois=[]
titles=[]
paper_venues=[]

## 2. Input

### 2.1 PDF Sources

In [6]:
pdf_urls = [
            'https://proceedings.mlr.press/v143/', #MIDL 2021
            'https://proceedings.mlr.press/v174/', #CHIL 2021
            'https://proceedings.mlr.press/v136/', #ML4H 2020
            'https://proceedings.mlr.press/v158/', #ML4H 2021
        ]

venue_labels = {
            'https://proceedings.mlr.press/v143/':'MIDL 2021',
            'https://proceedings.mlr.press/v174/':'CHIL 2021',
            'https://proceedings.mlr.press/v136/':'ML4H 2020',
            'https://proceedings.mlr.press/v158/':'ML4H 2021'
}


# To Do: move these under `ArticleScraper/ArticleScraper/spiders`

## 3. Crawler/Downloader

Make directory for pdfs if it does not exist

In [9]:
!mkdir data/pdfs

mkdir: data/pdfs: File exists


In [10]:
# scrapy code goes here
from scrapy.crawler import CrawlerProcess
from scrapy.http import Request

class MLRPRessCrawler(scrapy.Spider):
    """
    Spider for crawling PDFs from `proceedings.mlr.press`
    """
    name = "MLR_Press"

    def start_requests(self):
        urls = pdf_urls
        for url in urls:
            yield scrapy.Request(
                url=url, 
                meta={
                    "venue" : venue_labels[url]
                },
                callback=self.parse
            )

    def parse(self, response):
        for article in response.xpath('/html/body/main/div/div[*]'):
            try:
                yield Request(
                    url=article.xpath('p[3]/a[2]/@href').get(),
                    meta={
                        "title": article.xpath('p[1]/text()').get(),
                        "venue" : response.request.meta['venue']
                        },
                    callback=self.save_pdf
                )
            except Exception as e:
                print(e)

    def save_pdf(self, response):
        try:
            title = response.meta['title']+".PDF"
            venue = response.meta['venue']
            self.logger.info('Saving PDF %s', title)
            pdf_file = os.path.join('data','pdfs',venue+special_separator+title)
            with open(pdf_file, 'wb') as f:
                f.write(response.body)
            pdfs.append(pdf_file)
            titles.append(response.meta['title'])
            paper_venues.append(response.meta['venue'])
        except Exception as e:
            print(e)
            
crawler = CrawlerProcess({})
crawler.crawl(MLRPRessCrawler)
crawler.start()
# DOIs are unavailable for now
# pdf-files

2022-10-30 23:17:03 [scrapy.utils.log] INFO: Scrapy 2.6.3 started (bot: scrapybot)
2022-10-30 23:17:03 [scrapy.utils.log] INFO: Versions: lxml 4.9.1.0, libxml2 2.9.4, cssselect 1.1.0, parsel 1.6.0, w3lib 2.0.1, Twisted 22.8.0, Python 3.10.5 (v3.10.5:f377153967, Jun  6 2022, 12:36:10) [Clang 13.0.0 (clang-1300.0.29.30)], pyOpenSSL 22.1.0 (OpenSSL 3.0.5 5 Jul 2022), cryptography 38.0.1, Platform macOS-12.6-x86_64-i386-64bit
2022-10-30 23:17:03 [scrapy.crawler] INFO: Overridden settings:
{}
2022-10-30 23:17:03 [scrapy.utils.log] DEBUG: Using reactor: twisted.internet.selectreactor.SelectReactor
2022-10-30 23:17:03 [scrapy.extensions.telnet] INFO: Telnet Password: f2b02cf97fd10a70
2022-10-30 23:17:03 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.memusage.MemoryUsage',
 'scrapy.extensions.logstats.LogStats']
2022-10-30 23:17:03 [scrapy.middleware] INFO: Enabled downloader middlewares:
['

Request url must be str, got NoneType
Request url must be str, got NoneType
Request url must be str, got NoneType
Request url must be str, got NoneType


2022-10-30 23:17:04 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://proceedings.mlr.press/v136/uyttenhove20a/uyttenhove20a.pdf> (referer: None)
2022-10-30 23:17:04 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://proceedings.mlr.press/v136/wagh20a/wagh20a.pdf> (referer: None)
2022-10-30 23:17:04 [MLR_Press] INFO: Saving PDF Interpretable Epilepsy Detection in Routine, Interictal EEG Data using Deep Learning.PDF
2022-10-30 23:17:04 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://proceedings.mlr.press/v136/wang20a/wang20a.pdf> (referer: None)
2022-10-30 23:17:04 [MLR_Press] INFO: Saving PDF EEG-GCNN: Augmenting Electroencephalogram-based Neurological Disease Diagnosis using a Domain-guided Graph Convolutional Neural Network.PDF
2022-10-30 23:17:04 [MLR_Press] INFO: Saving PDF Confounding Feature Acquisition for Causal Effect Estimation.PDF
2022-10-30 23:17:05 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://proceedings.mlr.press/v136/weng20a/weng20a.pdf> (referer: N

## 4. PDF-Reader

Make directory for texts if it does not exist

In [11]:
!mkdir data/texts

mkdir: data/texts: File exists


In [12]:
# !ls data/pdfs

In [13]:
from pdfminer.high_level import extract_text

# i=0
for directory, subdirlist, filelist in os.walk('data/pdfs/'):
    # print(directory)
    for pdf in filelist:
        title = pdf[:-4]
        text_file = os.path.join('data','texts',title+'.txt')
        try:
            with open(text_file, 'w') as f:
                text_contents = extract_text(os.path.join(directory,pdf))
                f.write(text_contents)
                texts.append(text_file)
        except Exception as e:
            print(e)
            # print((i,text_file))
            # i+=1
# SEE ALSO these functions more layout-gnostic processing
# from pdfminer.high_level import extract_text_to_fp
# from pdfminer.layout import LAParams

No /Root object! - Is this really a PDF?


## 5. Saving Results

Combine venues (constant), titles , mention_matches and keyword_matches into final output.

In [14]:
merged_dict = {}

In [15]:

#UNUSED
#merged_dict['doi']=n/a
merged_dict['Venue']=paper_venues
merged_dict['Title']=titles

# TODO EXTRACT VENUE FROM TITLES using special_separator

# [titles.split(special_separator) for title in titles]


In [16]:
match_dataset = pd.DataFrame(
    merged_dict
)

In [17]:
# match_dataset= match_dataset.assign(Venue=venues[0])
# match_dataset= match_dataset.assign(DOI='n/a')

In [18]:
match_dataset

,Venue,Title
0,ML4H 2020,"Interpretable Epilepsy Detection in Routine, I..."
1,ML4H 2020,EEG-GCNN: Augmenting Electroencephalogram-base...
2,ML4H 2020,Confounding Feature Acquisition for Causal Eff...
3,ML4H 2020,Addressing the Real-world Class Imbalance Prob...
4,ML4H 2020,A Neural SIR Model for Global Forecasting
...,...,...
123,ML4H 2020,A Bayesian Hierarchical Network for Combining ...
124,MIDL 2021,Improved model-based deep learning for quantit...
125,ML4H 2021,Attention Distillation for Detection Transform...
126,ML4H 2021,End-to-End Sequential Sampling and Reconstruct...


In [19]:
match_dataset.to_csv('data/ResearchPapers.csv')

In [20]:
#WARNING: Preface.pdf